# 宿題シート: CartPole (倒立振子)


- このシートにはレポート課題に関する説明と，レポート課題の準備のための問題，そしてレポート課題の説明があります。
- このシート自体を提出する必要はありません。レポート課題については別のシートを見てください。

## CartPoleについて

- CartPole環境の説明: [openAI gym::CartPole v0](https://github.com/openai/gym/wiki/CartPole-v0)
- CartPoleのソース: [gym/gym/envs/classic_control/cartpole.py](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)
- 以下の説明は[vmayoral:basic_reinforcement_learning/tutorial4/README.md](https://github.com/vmayoral/basic_reinforcement_learning/blob/master/tutorial4/README.md)を参考資料として作成したものです。

## 準備: 基本モジュールの読み込み

以下のモジュール読み込みを行っておくこと。これまでのシートでは，モジュールを読み込む必要があることを明示するために、同じモジュールを複数のセルで何度も読み込む指示を与えていたものもあるが、このシートでは初めのセルで一度読み込むだけにする。

```python
import gym
from gym import wrappers
from IPython import display
import numpy as np
import random as ra
import math as ma

# 以下はjupyter notebookでアニメーション表示をするための設定
import matplotlib.pyplot as plt
%matplotlib inline
```

## Random action

以下はCartPole環境に対して、ランダムに行動選択を行う場合のプログラム例である。

FrozenLake環境では状態表示めに`env.render()`を使っていたが、CartPole環境での`env.render()`の出力は動画になり、そのままではjupyter notebook上には表示できない。そこで、jupyter notebook上に動画出力を行うための関数`render(env)`を以下のように用いる。

```python
def render(env):
        plt.imshow(env.render(mode='rgb_array')) #取得した画像をimshow()で画面表示バッファに入れる
        display.clear_output(wait=True) # 画面消去
        display.display(plt.gcf()) # 画面表示バッファの内容を画面に出力
        
env = gym.make('CartPole-v0')
for n_episode in range (2):
    s = env.reset()
    for t in range(100):
        render(env)
        a = env.action_space.sample() # ランダムに行動選択
        s, r, terminal, info = env.step(a)
        if terminal:
            print("Episode {} finished after {} timesteps".format(n_episode+1,t+1))
            break
```

## 環境情報
### state空間

FrozenLake環境等と同様に，openAI gym標準の方法でstate, actionに関する情報を取得できる。
```python
import gym
env=gym.make('CartPole-v0')

# 状態の情報
print("observation_space: ", env.observation_space)
print("observation_space.high: ",env.observation_space.high)
print("observation_space.low: ", env.observation_space.low)
```

### action空間

```python
print("action_space: ", env.action_space)
print("action_space.n: ", env.action_space.n)
```

### その他の情報

- `gym.spec.max_episode_steps`: 1エピソード内の最大ステップ数

## CartPole環境の問題点
- **問題点**: 状態空間が連続値をとる。
- **対策**: 連続値のまま扱う方法もあるが，今回は，状態空間を離散値に分割する。いくつの状態空間に分割するべきかは考慮すべき点。

**Q.** 状態空間をいくつかの離散値に分割することのメリットとデメリットを考えて説明しなさい。

## 状態の各要素値を離散化する方法

以下のNumpyの関数を使うと，連続値を簡単に離散化できる。

- `numpy.linspace(start,stop,num)`
    - `start`以上 `stop`以下の区間を`num`個の境界値で分割する(区間内のbin数は`num-1`，区間外も含めると`num+1`になる)
    - 返り値は，分割した区間の境界の値(`start`, `stop`を含む`num`個)を要素とする`ndarray`配列
- [`numpy.digitize(x,bins)`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.digitize.html)
    - 引数:
        - `x`: 離散化したい数値を要素とする一次元配列
        - `bins`: 境界値を要素とする配列(`ndarray`)
    - 返り値: 引数で渡した配列`x`の各要素`a`について，`bins[i-1] <= a < bins[i]`を満たす`i (i=1,...,len(bins)-1)`を配列にして返す。要素値が`[bins[0],bins[len(bins)-1]]`の範囲を超えていたら，その値に応じて0か最大値(`len(bins)`)となる。

サンプルプログラム
```python
border = np.linspace(start=-2.0, stop=2.0, num=9)

def to_bin(value, border): # valueを離散化し，そのインデックスを返す
    index=np.digitize(x=[value], bins=border)
    return index[0]
```

**Q.** 上記のサンプルプログラムを完成し，動作確認をしなさい。

**Q.** 以下のようなクラス`Digitize`をつくなさい。前問の内容を参考にし，`numpy.linspace()`と，`numpy.digitize()`を使うこと。

- 初期化メソッド:
    - 引数: 
        - `start, stop, num`:
            - `srart`以上`stop`以下の区間を`num`個の境界値で区切ることを指定するための引数
        - 各引数のデフォルト値は自由に決めて良い
    - 実行内容:
        - 値`start`から`stop`までの閉区間を`num`個の境界値で均等に分割し，その境界値を要素とする`ndarray`配列を，インスタンス変数`border`に代入する。
        - 注) `start`未満および，`stop`以上の区間も含めると，全部で`num+1`の区間に分割することになる
- インスタンス・メソッド
    - 名前: `bin`
    - 引数:
        - `value`: スカラー値
    - 返り値
        - 引数`value`が，インスタンス変数`border`によって決まる区間のどこに入るか，そのインデックスを返す。インデックスの値は，前問と同様。


**動作確認:** 以下を確認しなさい。

1. 以下を実行する

```python
class Digitize:
    """
    この部分は自分で作る
    """
state_digit=Digitize(start=-2.0, stop=2.0, num=9)
```
2. `state_digit.border==[-2., -1.5, -1., -0.5,  0.,0.5, 1.,1.5, 2. ])`であることを確認
3. `len(state_digit.border)==9`であることを確認
4. `state_digit.val(-2.1)==0`であることを確認
5. 以下の各値がどうなるべきかを考え，実際にその値であることを確認
```python
state_digit.val(-2.1)
state_digit.val(-1.9)
state_digit.val(1.9)
```

**Q.** cartpole環境の状態の各要素をそれぞれ10個の離散値にした場合，とりうる状態数は全部でいくつになるだろう?

## Q学習

### QlearnerV2
**課題**

以下の設問に沿って CartPole環境用の学習プログラムを作成しなさい。

**Q1.** 課題6-classにおいてFrozenLake環境用に作成したクラス`Qlearner`を次のセルにコピペし、以下のように修正しなさい。

**注意:** クラス`Qlearner`は、第一回レポート用**ではなく**、**課題6-classの指示に従って作成したもの**(バグはとっておく)を使うこと。

**修正点**

- クラス名: `QlearnerV2`にする
- 引数として受け取った状態変数をすでに作成したクラス`Digitize()`により，離散化する。
    - **状態分割を行う区間は各自検討**すること。
    - 離散化した4つの状態値を，Q関数の引数に与えるリストにするために，以下の関数`build_state()`を用いる
- 関数`learning()`に、**引数`terminal=False`を追加**。この引数は現時点では処理には使わないが、各自必要に応じて利用して良い。

以下に`class Qlearner`の**修正が必要な点のみ**を示す。

```python
index_list = lambda L, value: [i for i,x in enumerate(L) if x==value]

def build_state(states): # 状態の各要素値のリスト[x,v,p,q]を文字列xvpqにする
    return int("".join(map(lambda state: str(int(state)), states)))

class QlearnerV2: # 名前を変える!
    def __init__(self, action_list, epsilon=0.1, alpha=0.2, gamma=0.9, q0=0.0):

        # 中略
        
        N_BINS=10
        self.dig_x=Digitize(start=???, stop=???, num=N_BINS-1)# ??? に入れる値は自分で決める
        self.dig_v=Digitize(start=???, stop=???, num=N_BINS-1)
        self.dig_ang=Digitize(start=???, stop=???, num=N_BINS-1)
        self.dig_vang=Digitize(start=???, stop=???, num=N_BINS-1)

    def reset(self): # 処理内容は各自記入
        
    def digitize(self,state): # 状態stateの各要素を離散化して、一つの文字列にする関数
        x, v, ang, v_ang=state # タプルstateの各要素が状態値
        return(build_state([self.dig_x.val(x), # 各状態値を離散化して文字列化
                     self.dig_v.val(v),
                     self.dig_ang.val(ang),
                     self.dig_vang.val(v_ang)]))
    def get_q(self, s, a): # 処理内容は各自記入

    def get_maxQ(self,s): # 同上

    def get_action(self, s):
        s=self.digitize(s)
        # 以下略
        
    def learning(self, s1, a1, r, s2, terminal=False): # 引数terminal追加
        '''
        Q-learning: Q(s, a) += alpha * (r + gamma*max(Q(s')) - Q(s,a))
        '''
        # get_action()を参考にs1,s2を離散化する命令文を追加する
```

### play_env_v2()
**Q2.** シート`7-frozenlake`で作成した関数`play_env()`を修正して，状態値の離散化を行えるようにした関数`play_env_v2()`を作り，`Qlearner`によるCartPole環境の学習を行えるようにしなさい。

- **変更点**は以下の通り
    1. `env.render()`は以下のサンプルプログラムにある関数`render(env,i_episode,t)`の呼び出しに変更する。(理由は前述)
    2. **引数の`max_step`は廃止**し、関数内の`max_step`は全てクラスオブジェクト`env`のインスタンス変数`env.spec.max_episode_steps`に置き換える。
    3. 変数`history_steps`, `history_score`には**NumPyの`ndarray`を使う**
    4. **変数`history_score`は、エピソード成功のときに1, 失敗のとき0**になるよう変更

全体像は以下のようになる。
```python
def render(env,i_episode,t): # jupyter notebookに動画を表示するための関数
    plt.imshow(env.render(mode='rgb_array'))
    display.clear_output(wait=True)
    display.display(plt.gcf())
    print("Episode={}, Time={}".format(i_episode+1,t+1))    

# 引数のデフォルト値は、まずは以下の値で試す
def play_env_v2(env, agent, max_episode=2000, learning=True, render_step=100, verbose=False):
    history_step=... # 要素数max_episodeのndarrayを定義。各要素の初期値は0にする
    history_score=.. # 同上

    # 以下は自分で書く(env.render()とhistory_scoreの設定変更忘れずに)

# いよいよ実行!
env=gym.make('CartPole-v0')
env=wrappers.Monitor(env, './video',force=True) # ディレクトリvideoにアニメーションを保存するための設定

Qagent=QlearnerV2(action_list=list(range(env.action_space.n)), 
                   epsilon=0.1, alpha=0.5, gamma=0.9, q0=0.0)
history_step,history_score=play_env_v2(env,agent=Qagent,verbose=False)
```

**ビデオのファイル出力について**

CartPoleを使うための宣言は`env=gym.make('CartPole-v0')`であるが，上記のように`wrappers.Monitor()`を追加するとmp4で適宜保存してくれるので，学習の途中経過をあとで動画で確認できる。
保存タイミングは，env.step()を呼び出した回数が1000回以下のときには$0, 1^3, 2^3, 3^3,\cdots$と$n^3$回ごと，それ以降は1000回ごとになる。

### 学習結果の表示
上述のプログラムが無事動いたかは以下で確認する。

1. エラーはないか? ただし以下の警告文は無視して良い
```
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
```
2. Matplotlibで, `play_env_cont()`の返り値の`history_step`や`history_score`を表示する。シート`6-class`で作成した`plot_history()`を使うのも良い。
3. `./video`にできたビデオで確認する。

**Q3.** 上記までで作ったプログラムでは、残念ながら**学習性能は低い**。何故かよく考え、まずは、報酬条件の変更を試みることで、学習性能を向上させなさい。

- 報酬設定を変えるときには，エージェントのクラス`QlearnerV2`のインスタンス・メソッド`learning()`を修正し，引数で与えられる状態や報酬の情報をもとに再設定しなさい。
- 詳しくはシート`7-frozenlake`の3.2参照。クラス`QlearnerV2`の派生クラスを作ると便利。

## Actor-Critic法

**Q.** 上記のクラス`QlearnerV2`を改造して、Actor-Critic法による学習エージェントのクラス`ActorCritic`を作りなさい。変更点は以下になる。

- 初期化メソッドの変更
    - 引数: `v0`を追加(状態価値関数Vの初期値)
    - 処理内容
        - インスタンス変数`v0`に引数`v0`の値を代入
        - 価値関数$V(s)$の値を格納する空の辞書`V`を準備(Qlearnerの辞書`Q`の初期化と同様)
- インスタンスメソッド`reset()`の修正
    - 辞書`V`の要素を空にリセットする(辞書`Q`のリセットと同様)
- インスタンスメソッド`get_V()`の追加
    - 引数: `s` (状態)
    - 返値: 辞書`V`を参照し、価値関数$V(s)$の値を返す。引数で与えられた状態`s`が辞書Vのキーにないときには、インスタンス変数`v0`の値を返す。(すでにある`get_Q()`とほぼ同様の内容)
- インスタンスメソッド`learning()`の修正
    - Q学習をActor-Critic法に変更する
        $$\delta=r+\gamma V(s')-V(s)$$
        $$V(s) \leftarrow V(s)+\alpha\delta$$
        $$p(s, a) \leftarrow p(s,a) + \alpha\delta$$
    - $p(s,a)$は、状態$(s,a)$における行動を決定するための関数
        - 新たに辞書`p`を作ることはせず、`Qlearner`で使っていた辞書`Q`をそのまま使うことにする。行動決定も`get_action()`をそのまま利用可能
    - $p(s,a)$の学習定数$\alpha$は、$V(s)$の学習定数とは異なる値にすることもあるが、とりあえず同じ値にしておく

**Q.** Actor-Critic法による学習は、Q学習に比べて学習性能が良いかどうか検討しなさい。

## softmax法
**Q.** 以下のような関数`softmax_rand()`を作りなさい。

- 引数:
    - `V`: `ndarray`配列
    - `T=1`: softmax法のパラメータ
- 返値:
    - 引数`V`の各要素値に対してsoftmax法を適用した確率で、そのインデックスをランダムに返す。
    - 例) 引数が`l=[1,3,4]`の場合は，ランダムに各要素のインデックスを返すが、インデックス`k`が選択される確率$P(k)$ ($k=0,1,2$)は次式の通り。
    
    $$\displaystyle P(k)=\frac{e^\frac{V(k)}{T}}{\displaystyle\sum_{i=0}^{N-1}e^\frac{V(i)}{T}}$$

- **動作確認:** 以下を確認しなさい。
    - `softmax_rand(l=[1,1])`の返り値は0か1であり、その比率は1:1である。
    - `softmax_rand(l=[0,log2,1])`の返り値は0か1か2であり、その比率は1:2:eである。

**Q2.** 前問で作った`softmax_rand()`の動作確認のため以下を行いなさい。

1. 動作確認用のための以下のような関数`test()`を作りなさい。
        - 引数
            - `l`: リスト(要素は数値)
            - `T=1`: softmax法のパラメータ
            - `num`: 確率発生回数
        - 動作
            - 引数`l`,`T`を`softmax_rand()`に渡し、`num`回乱数を発生させる。その乱数値が、引数`l`の各要素のインデックスに何回該当したかを表示する。
2. 関数`test()`を使って以下を確認しなさい。
    - `softmax_rand(l=[1,1],T=1)`の返り値は0か1であり、その比率はほぼ1:1である。
    - `softmax_rand(l=[0,log2,1],T=1)`の返り値は0か1か2であり、その比率はほぼ1:2:eである。

**Q.** クラス`ActorCritic`の行動選択メソッド`get_action()`はε-greedy法になっているはずである。

1. 行動選択をsoftmax法に変更したクラス`ActorCriticSofxmax`を、クラス`ActorCritic`の派生クラスとして作りなさい。もしくは、クラス`ActorCritic`の初期化メソッドで、ε-greedy法とsoftmax法を切り替えられるようにしなさい。
2. softmax法の場合の学習効率はε-greedy法と比べて良いだろうか。softmax法のパラメータ$T$はどのような値が良いのだろう。特に良い値があるとしたら、その理由はどうしてだろう。

[課題]
- 報酬の設定方法次第で学習効率はいろいろ変わる。例えば，タスク失敗時の報酬を-5とすると，どのように学習速度は変わるだろうか。
- 離散化による状態分割数は多くするのと少なくするのとどちらが良いだろうか
- 学習の結果，どのような行動価値関数が獲得されただろうか
- 行動の決定に，greedy法，ε-greedy法，softmax法のどれを用いるのがいいのだろうか
- Actor-Critic法を使うと，学習速度は改善するだろうか
- 学習速度を上げるには他にどのような工夫をできるだろうか。報酬設定だけではなく，いろいろな可能性を考えて検証しなさい。